In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import time
import os
import logging

# Initialize logger for incident response actions
logging.basicConfig(level=logging.INFO, filename='incident_response.log', format='%(asctime)s - %(message)s')

# Example incident dataset (incident_id, incident_type, severity, source, timestamp)
incident_data = [
    {'incident_id': 1, 'incident_type': 'DDoS', 'severity': 8, 'source': '192.168.1.1', 'timestamp': time.time()},
    {'incident_id': 2, 'incident_type': 'Phishing', 'severity': 5, 'source': '192.168.1.2', 'timestamp': time.time()},
    {'incident_id': 3, 'incident_type': 'Malware', 'severity': 7, 'source': '192.168.1.3', 'timestamp': time.time()},
    {'incident_id': 4, 'incident_type': 'Brute Force', 'severity': 6, 'source': '192.168.1.4', 'timestamp': time.time()},
    {'incident_id': 5, 'incident_type': 'Ransomware', 'severity': 9, 'source': '192.168.1.5', 'timestamp': time.time()},
]

# Convert incident data to DataFrame
def preprocess_incidents(data):
    df = pd.DataFrame(data)
    # Convert timestamp to a more useful feature (time of day)
    df['time_of_day'] = pd.to_datetime(df['timestamp'], unit='s').dt.hour
    return df

# Feature engineering for machine learning model
def extract_features(df):
    df['incident_type'] = df['incident_type'].astype('category').cat.codes  # Convert types to numeric
    X = df[['incident_type', 'severity', 'time_of_day']].values
    y = np.random.randint(0, 2, size=(len(df)))  # 0 = low priority, 1 = high priority (binary classification)
    return X, y

# Train the incident prioritization model
def train_incident_model(data):
    df = preprocess_incidents(data)
    X, y = extract_features(df)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_scaled, y)
    return model, scaler

# Classify incidents and assign priority
def classify_incidents(model, scaler, data):
    df = preprocess_incidents(data)
    X, _ = extract_features(df)
    X_scaled = scaler.transform(X)
    
    priorities = model.predict(X_scaled)
    df['priority'] = priorities
    return df

# Automated Response to high-priority incidents
def automated_response(incident):
    priority = incident['priority']
    source_ip = incident['source']
    
    if priority == 1:
        logging.info(f"High priority incident detected: {incident['incident_type']} from {source_ip}")
        # Simulating automated response actions
        os.system(f"iptables -A INPUT -s {source_ip} -j DROP")  # Block the source IP
        logging.info(f"Blocked IP: {source_ip}")
        # Send alert (simplified for demo)
        print(f"Alert: High-priority incident detected! Source IP {source_ip} blocked.")
    else:
        logging.info(f"Low priority incident detected: {incident['incident_type']} from {source_ip}")
        print(f"Low priority incident detected: {incident['incident_type']} from {source_ip}")
        
# Real-time Incident Response Monitoring
def monitor_and_respond(incident_data):
    model, scaler = train_incident_model(incident_data)
    
    while True:
        # Simulate new incoming incidents
        new_incidents = [
            {'incident_id': 6, 'incident_type': 'DDoS', 'severity': 8, 'source': '192.168.1.6', 'timestamp': time.time()},
            {'incident_id': 7, 'incident_type': 'Phishing', 'severity': 3, 'source': '192.168.1.7', 'timestamp': time.time()},
        ]
        
        # Classify the incidents based on priority
        classified_incidents = classify_incidents(model, scaler, new_incidents)
        
        for _, incident in classified_incidents.iterrows():
            automated_response(incident)
        
        time.sleep(30)  # Check for new incidents every 30 seconds

# Main function to start monitoring and responding
if __name__ == "__main__":
    monitor_and_respond(incident_data)